In [1]:
import tensorflow as tf
import pandas as pd
from gensim.models import Word2Vec, KeyedVectors
from preprocessing import process_predict_data
import random

with open("save/model.tf", mode='rb') as f:
        graph_def = tf.GraphDef()
        try:
            graph_def.ParseFromString(f.read())
        except:
            print('try adding PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python ' +
                  'to environment.  e.g.:\n' +
                  'PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python ipython\n' +
                  'See here for info: ' +
                  'https://github.com/tensorflow/tensorflow/issues/582')

with open('intent.txt') as f:
    intent = [(idx, l.strip()) for idx, l in enumerate(f.readlines())]


g = tf.Graph()
with tf.Session(graph=g) as sess, g.device('/cpu:0'):
    tf.import_graph_def(graph_def, name='cnn')
    names = [op.name for op in g.get_operations()]
print("initialized intent model with structure")
print()
for name in names:
    print(name)
    
# load word2vec model
model = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin",binary = 'True')
#model = Word2Vec.load("trainedWord2vecmodel")   

Using TensorFlow backend.


initialized intent model with structure

cnn/X
cnn/conv/stack/0/W
cnn/conv/stack/0/W/read
cnn/conv/stack/0/conv
cnn/conv/stack/0/b
cnn/conv/stack/0/b/read
cnn/conv/stack/0/H
cnn/conv/stack/0/relu
cnn/conv/stack/0/pool
cnn/conv/stack/1/W
cnn/conv/stack/1/W/read
cnn/conv/stack/1/conv
cnn/conv/stack/1/b
cnn/conv/stack/1/b/read
cnn/conv/stack/1/H
cnn/conv/stack/1/relu
cnn/conv/stack/1/pool
cnn/conv/stack/2/W
cnn/conv/stack/2/W/read
cnn/conv/stack/2/conv
cnn/conv/stack/2/b
cnn/conv/stack/2/b/read
cnn/conv/stack/2/H
cnn/conv/stack/2/relu
cnn/conv/stack/2/pool
cnn/preFc/concat/axis
cnn/preFc/concat
cnn/preFc/Reshape/shape
cnn/preFc/Reshape
cnn/dropout/dropout/keep_prob
cnn/dropout/dropout/Shape
cnn/dropout/dropout/random_uniform/min
cnn/dropout/dropout/random_uniform/max
cnn/dropout/dropout/random_uniform/RandomUniform
cnn/dropout/dropout/random_uniform/sub
cnn/dropout/dropout/random_uniform/mul
cnn/dropout/dropout/random_uniform
cnn/dropout/dropout/add
cnn/dropout/dropout/Floor
cnn/dropout/d

In [2]:
import random
def get_intent(text):
    global g
    global intent
    global model
    x = g.get_tensor_by_name(names[0] + ':0')
    softmax = g.get_tensor_by_name(names[-1] + ':0')
    seq_len = x.get_shape().as_list()[1]
    df = pd.DataFrame(index=[0], columns=["text"])
    df["text"][0] = text
    data_x = process_predict_data(df,model,seq_len)
    
    with tf.Session(graph=g) as sess:
        res = softmax.eval(feed_dict={x: data_x,'cnn/dropout/dropout/keep_prob:0': 1})#dropout keep =100%
    print (res[0])
    return intent[res[0].argmax()][1], max(res[0])

In [3]:
def get_response(text, thresh):
    intent, prob = get_intent(text)
    print (intent, prob)
    #if prob >= thresh:
        #res_file = "response/"+intent+".txt"
        #file = open(res_file)
        #response_list = file.read()
        #response_list = response_list.split("\n")
        #print(response_list[random.randint(0,len(response_list)-2)])

In [6]:
get_response("professor's name",0.5)

[  1.32176003e-09   2.17942175e-09   1.00000000e+00   4.91788965e-09
   1.46920087e-10   4.40407022e-14   1.45501877e-08   1.61797313e-08
   3.34470207e-09   1.73359157e-12   1.70097037e-18   2.64165487e-14
   4.82446913e-21   7.99403710e-10   7.55868655e-15]
Professor's name 1.0
